# Credit card fraud detector combining Fraud Detector and SageMaker models

## Investigate and process the data

Let's start by downloading and reading in the credit card fraud data set.
The dataset has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection.

In [ ]:
%%bash
wget https://fraud-detector-blog-assets.s3.amazonaws.com/creditcard.csv

In [1]:
from datetime import datetime
import numpy as np 
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

data = pd.read_csv('creditcard.csv', delimiter=',')

Let's take a peek at our data (we only show a subset of the columns in the table):

In [ ]:
print(data.columns)
data.describe()

The class column corresponds to whether or not a transaction is fradulent. We see that the majority of data is non-fraudulent with only $492$ ($.173\%$), check the Class column mean, of the data corresponding to fraudulent examples.

Checking the mean and standard deviation of the features.

In [ ]:
import matplotlib.pyplot as plt

data.hist(bins=50,figsize=(20,15))
plt.show()

Looks good, columns 𝑉𝑖 have been normalized to have 0 mean and unit standard deviation as the result of a PCA. Now, lets change the data to be Amazon Fraud Detector compatible.

In [2]:
# to lowecase
data.columns = map(str.lower, data.columns)
print(data.columns)

Index(['time', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'amount', 'class'], dtype='object')


In [3]:
def standardize_headers(x):
    if any(char.isdigit() for char in x):
        if int(x[1:]) > 26:
            return 'va'+chr(int(x[1:])+70)
        return 'v'+chr(int(x[1:])+96)
    return x

# mapping number to letter
data.rename(columns=standardize_headers, inplace=True)
print(data.columns)

Index(['time', 'va', 'vb', 'vc', 'vd', 've', 'vf', 'vg', 'vh', 'vi', 'vj', 'vk', 'vl', 'vm', 'vn', 'vo', 'vp', 'vq', 'vr', 'vs', 'vt', 'vu', 'vv', 'vw', 'vx', 'vy', 'vz', 'vaa', 'vab', 'amount', 'class'], dtype='object')


Then change the timestamp and label column names

In [4]:
# rename to the Fraud Detector name conventions 
data.rename(columns={'time':'timedelta'}, inplace=True)
print(data.columns)

Index(['timedelta', 'va', 'vb', 'vc', 'vd', 've', 'vf', 'vg', 'vh', 'vi', 'vj', 'vk', 'vl', 'vm', 'vn', 'vo', 'vp', 'vq', 'vr', 'vs', 'vt', 'vu', 'vv', 'vw', 'vx', 'vy', 'vz', 'vaa', 'vab', 'amount', 'class'], dtype='object')


Get epoch time for the initial dataset date

In [10]:
from datetime import datetime

epoch = datetime.utcfromtimestamp(0)
def unix_time_seconds(dt):
    return (dt - epoch).total_seconds()

# Lets pretend that the data is from 2 days ago and we can test at the end with todays date.
start_dt = datetime.strptime('Aug 4 2020  12:00AM', '%b %d %Y %I:%M%p')
start_dt = datetime.now()
start_ep = unix_time_seconds(start_dt)
print(start_ep)

1596600133.898222


Translate the current timestamp format (increasing seconds) to ISO 8601 standard

In [ ]:
import time

def to_datetime(x):
    current_ep = start_ep + x
    current_dt = time.strftime('%Y-%m-%dT%H:%M:%SZ', time.localtime(current_ep))
    return current_dt

# translate seconds delta to actual datetimes in ISO 8601
data['EVENT_TIMESTAMP'] = data['EVENT_TIMESTAMP'].apply(to_datetime)

data.head()

We will split our dataset to get some test samples. After will upload the data to S3 using boto3.

In [29]:
msk = np.random.rand(len(data)) < 0.995
test = data[~msk]
len(test)

1414

Uploading the data for training

In [ ]:
from io import StringIO # python3; python2: BytesIO 
import boto3

bucket = 'sample-creditcard-dataset' # already created on S3
csv_buffer = StringIO()
data.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'creditcard-fraud-detector-training.csv').put(Body=csv_buffer.getvalue())

Uploading the data for testing

In [ ]:
from io import StringIO # python3; python2: BytesIO 
import boto3

bucket = 'sample-creditcard-dataset' # already created on S3
csv_buffer = StringIO()
test.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'creditcard-fraud-detector-test.csv').put(Body=csv_buffer.getvalue())

Once we have the datasets ready we need create the necesary entities for build and deploy the fraud detection model. This can be done within the Amazon Fraud Detector console or through the API as shown in the second jupyter notebook.

Testing our model endpoint

In [6]:
import boto3
# -- fraud detector client --
client = boto3.client('frauddetector',)

Entity, Detector, Model, and File Information

In [7]:
ENTITY_TYPE    = "transaction" #change to your entity
EVENT_TYPE     = "testevent2" #change to your envent_type

DETECTOR_NAME = "model_ensemble" #change to your detector
DETECTOR_VER  = "1"

# -- name and version of model, used to get the model column names -- 
MODEL_NAME    = "fraud_model"
MODEL_VER     = "1"

record_count = 1000

Cleaning the test dataset from training columns and defining the start datetime.

In [30]:
model_variables = [column for column in test.columns if column not in  ['class']]
dateTimeObj = datetime.strptime('Sep 3 2013  12:00AM', '%b %d %Y %I:%M%p')
#dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%Y-%m-%dT%H:%M:%SZ")
print(' '.join(model_variables))
cols = ['timedelta']
#test[cols].applymap(np.int64)
test.head()

timedelta va vb vc vd ve vf vg vh vi vj vk vl vm vn vo vp vq vr vs vt vu vv vw vx vy vz vaa vab amount


,timedelta,va,vb,vc,vd,ve,vf,vg,vh,vi,vj,vk,vl,vm,vn,vo,vp,vq,vr,vs,vt,vu,vv,vw,vx,vy,vz,vaa,vab,amount,class
445,323.0,-0.704133,0.341397,1.740027,-1.661595,0.872313,-0.007311,0.923083,-0.575939,0.447697,0.264465,0.302329,-0.589641,-0.979096,-0.414870,0.227651,0.863539,-1.560955,0.570082,0.262597,0.018389,-0.206984,-0.321045,-0.334626,-0.813176,-0.265089,0.689043,-0.904113,-0.579831,0.77,0
493,363.0,-1.028699,0.910515,1.915180,2.469384,-0.008375,0.597584,0.251531,-0.331730,-0.095639,1.348750,-1.126126,-0.056429,1.141532,-0.984674,0.409776,0.291837,-0.648132,0.337319,0.242736,-0.186373,0.204878,0.948674,-0.014794,-0.064869,-0.882317,0.022126,-0.673108,0.085784,37.92,0
727,549.0,0.033854,-1.818313,1.077334,3.350537,-1.292195,1.546080,-0.282520,0.402055,0.928263,-0.011706,-1.801164,0.092265,-0.871234,-0.757440,-1.720001,-0.204210,0.428464,-0.689307,-0.677380,0.809245,0.108768,-0.451008,-0.475113,-0.261082,0.181753,0.025919,-0.019035,0.117564,530.85,0
1172,914.0,-0.820178,1.225605,1.517290,-0.007492,0.310123,-0.936490,1.026234,-0.163058,-0.500997,-0.424367,-0.463979,0.220510,0.481360,0.021309,0.296479,0.193774,-0.703515,-0.168465,-0.955137,0.015005,0.032831,0.136686,-0.347711,0.390412,0.849821,-0.405567,0.074595,0.040960,11.03,0
1228,947.0,1.130882,-0.306948,0.998749,0.225915,-0.544978,0.872664,-0.938950,0.372366,0.459415,-0.015577,0.839393,0.964977,0.923658,-0.166225,1.159326,0.983879,-1.002706,0.659583,-0.338309,0.024383,0.261644,0.792110,-0.145284,-0.763391,0.212113,0.638136,0.024789,0.013678,28.90,0


In [27]:
import uuid 

# test the endpoint with a single prediction.
eventId = uuid.uuid1()
testrecord = test[model_variables].head(1).astype(str).to_dict(orient='records')[0]
pred = client.get_event_prediction(detectorId=DETECTOR_NAME, 
                                       detectorVersionId=DETECTOR_VER,
                                       eventId = str(eventId),
                                       eventTypeName = EVENT_TYPE,
                                       eventTimestamp = timestampStr, 
                                       entities = [{'entityType': ENTITY_TYPE, 'entityId':str(eventId.int)}],
                                       eventVariables=  testrecord)
print(pred)

{'timedelta': '46'}
{'modelScores': [{'modelVersion': {'modelId': 'sagemaker_compatible', 'modelType': 'ONLINE_FRAUD_INSIGHTS', 'modelVersionNumber': '1.0'}, 'scores': {'sagemaker_compatible_insightscore': 155.0}}], 'ruleResults': [{'ruleId': 'low_fraud_risk', 'outcomes': ['allow_transaction']}], 'ResponseMetadata': {'RequestId': '0b886067-e493-460c-b043-8b9c89b5519e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Wed, 05 Aug 2020 04:51:00 GMT', 'x-amzn-requestid': '0b886067-e493-460c-b043-8b9c89b5519e', 'content-length': '262', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


The next block will use some parallelization to run several test against the fraud detector endpoint.

In [ ]:
import dask 
from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:90% }</style>"))

start = time.time()

@dask.delayed
def _predict(record):
    eventId = uuid.uuid1()
    try:
        pred = client.get_event_prediction(detectorId=DETECTOR_NAME, 
                                       detectorVersionId=DETECTOR_VER,
                                       eventId = str(eventId),
                                       eventTypeName = EVENT_TYPE,
                                       eventTimestamp = timestampStr, 
                                       entities = [{'entityType': ENTITY_TYPE, 'entityId':str(eventId.int)}],
                                       eventVariables=  record) 
        
        record["score"]   = pred['modelScores'][0]['scores']["{0}_insightscore".format(MODEL_NAME)]
        record["outcomes"]= pred['ruleResults'][0]['outcomes']
        return record
    
    except:
        pred  = client.get_event_prediction(detectorId=DETECTOR_NAME, 
                                       detectorVersionId='1',
                                       eventId = str(eventId),
                                       eventTypeName = EVENT_TYPE,
                                       eventTimestamp = timestampStr, 
                                       entities = [{'entityType': ENTITY_TYPE, 'entityId':str(eventId.int)}],
                                       eventVariables=  record) 
        record["score"]   = "-999"
        record["outcomes"]= "error"
        return record

#just testing with 100 samples, increase the record_count variable o remove the .head to test the entire test dataset
predict_data  = test[model_variables].head(record_count).astype(str).to_dict(orient='records')
predict_score = []

i=0
for record in predict_data:
    clear_output(wait=True)
    rec = dask.delayed(_predict)(record)
    predict_score.append(rec)
    i += 1
    print("current progress: ", round((i/record_count)*100,2), "%" )
    
predict_recs = dask.compute(*predict_score)

# Calculate time taken and print results
time_taken = time.time() - start
tps = len(predict_recs) / time_taken

print ('Process took %0.2f seconds' %time_taken)
print ('Scored %d records' %len(predict_recs))

In [ ]:
predictions = pd.DataFrame.from_dict(predict_recs, orient='columns')
predictions.head(record_count)
predictions.loc[predictions['score'] >=950, 'vaa':'outcomes']

See the model metrics on CloudWatch and the prediction history in Fraud Detector.

In [ ]:
# save the results to a csv file
predictions.to_csv(MODEL_NAME + "precictions.csv", index=False)

In [ ]:
#data.loc[data['vaa'] == 0.14205158164005, 'vaa':'EVENT_LABEL']

In [ ]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('creditcard.csv', delimiter=',')

feature_columns = df.columns[:-1]
label_column = df.columns[-1]

features = df[feature_columns].values.astype('float32')
labels = (df[label_column].values).astype('float32')

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.1, random_state=42)

payload = ','.join(map(str, X_train[0]))

print(payload)

In [ ]:
import json

sagemaker_endpoint_name = 'fraud-detection-endpoint'
sagemaker_runtime = boto3.client('sagemaker-runtime')
response = sagemaker_runtime.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='text/csv',
                                                 Body=payload)
print(response)
pred_proba = json.loads(response['Body'].read().decode())
formatted_float = "{:.10f}".format(pred_proba)
prediction = 0 if pred_proba < 0.5 else 1
# Note: XGBoost returns a float as a prediction, a linear learner would require different handling.
print("classification pred_proba: {}, prediction: {}".format(formatted_float, prediction))

Finish